# Arabc Characters MNIST 
A [Kaggle Project](https://www.kaggle.com/mloey1/ahcd1) provided by [Benha University](http://bu.edu.eg/staff/mloey)

Project By: [Uzair](https://github.com/Uzair05)

## Using CSV Datasets

### Download and move data

In [ ]:
#!pip install --upgrade kaggle
!kaggle datasets download -d mloey1/ahcd1

In [ ]:
import os 
import zipfile

try:
    os.mkdir("./data/")
except Exception as err:
    print(f"Error in creating data/:\t{err}")


localzip = "./ahcd1.zip"
zip_ref = zipfile.ZipFile(localzip, 'r')
try:
    zip_ref.extractall('data/')
except Exception as err:
    print(f"Erorr in zip extraction:\t{err}")
finally:
    zip_ref.close()

### Handle Data
Create test, train and validation data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
#import random as rand

In [ ]:
data = pd.read_csv("./data/csvTestImages 3360x1024.csv")
label = pd.read_csv("./data/csvTestLabel 3360x1.csv")

In [ ]:
x_train = data.to_numpy()
x_shape, _ = x_train.shape
x_train = x_train.reshape(x_shape, 32, 32)
y_train = label.to_numpy()
y_shape, _ = y_train.shape
y_train = y_train.reshape(y_shape,)

In [ ]:
for i in range(x_shape):
    x_train[i,:,:] = np.rot90(x_train[i,:,:], axes=(1,0))

In [ ]:
plt.imshow(x_train[6,:,:])

### Create Model

In [ ]:
# Model

### Train Model

### Evaluate Model

### Save Model

## Using Images Datasets

### Handle Data

#### Bash Command - Linux


* bash script to sort labelled images into directories.  
    * Use directories in imageneration

#### Python Command - Windows

### CNN Transfer Learning
